**Prepare data for usage**

In [1]:
import pandas as pd
import numpy as np
import io
import requests
import os
import zipfile
import urllib.request
import matplotlib.pyplot as plt
# Set directory where files are downloaded to. Chdir has to be changed in order to run on another computer
os.chdir('C:\\Users\justu\\Desktop\\Masterarbeit\\Data\\CE') 
os.getcwd()

'C:\\Users\\justu\\Desktop\\Masterarbeit\\Data\\CE'

**Step 1**: Download data.
Note that the data is compressed as zip files on the website. Download the whole data for the relevant years. This could take a couple of minutes

In [2]:
y = [0,1,7,8,9]

for y_val in y:
    urllib.request.urlretrieve(f'https://www.bls.gov/cex/pumd/data/comma/intrvw0{y_val}.zip', f'intrvw0{y_val}.zip') 

**Step 2**: Generate expenditure data for the given years.
2.1: Generate exp data for each year seperately

In [2]:
y = [0,1,7,8,9] # renew years

ID = ['NEWID'] #identifier

# Expenditure: define the different expenditure types in line with Misra & Surico. 
# It doesn't matter in which quarter of the year the consumption took place.
FD = ['FDHOMECQ', 'FDHOMEPQ', 'FDAWAYCQ', 'FDAWAYPQ', 'ALCBEVCQ', 'ALCBEVPQ' ] # food
SND = FD + ['UTILCQ', 'UTILPQ', 'HOUSOPCQ', 'HOUSOPPQ', 'PUBTRACQ', 'PUBTRAPQ', 'GASMOCQ', 
            'GASMOPQ', 'PERSCACQ', 'PERSCAPQ', 'TOBACCCQ', 'TOBACCPQ', 'MISCCQ', 'MISCPQ'] # strictly non-durables
ND = SND + [ 'APPARCQ', 'APPARPQ', 'HEALTHCQ', 'HEALTHPQ', 'READCQ', 'READPQ'] # non-durables
DUR = ['EDUCACQ', 'EDUCAPQ', 'HOUSEQCQ', 'HOUSEQPQ', 'SHELTCQ', 'SHELTPQ', 'ENTERTCQ', 'ENTERTPQ',
       'CARTKNCQ', 'CARTKNPQ', 'OTHVEHCQ', 'OTHVEHPQ', 'MAINRPCQ', 'MAINRPPQ', 'VEHINSCQ', 'VEHINSPQ', 
       'VRNTLOCQ', 'VRNTLOPQ', 'VEHFINCQ', 'VEHFINPQ'] # durables

TIME = ['QINTRVMO', 'QINTRVYR'] # relevant time variables

DEMO = ['AGE_REF', 'FAM_SIZE', 'ST_HOUS', 'MARITAL1', 'AGE2', 'PERSLT18', 'RESPSTAT', 'CUTENURE', 'FINCBTAX'] # relevant demographics available for all years
DEMO2 = ['FSALARYM', 'FINCBTXM'] # relevant demographics available for the second stimulus only

ASSETS = ['SAVACCTX','SAVA_CTX','CKBKACTX', 'CKBK_CTX', 'SECESTX', 'SECESTX_' ] # saving accounts; checking accounts, brookerage accounts

    # 'RESPSTAT' info on completeness of income information
fiscal = dict()

for y_val in y:
    zf = zipfile.ZipFile(f'{os.getcwd()}/intrvw0{y_val}.zip', 'r')
    with zf as zipobj:
        df = zipobj.namelist()
    fmli = [s for s in df if 'fmli' in s]
    zf = zipfile.ZipFile(f'{os.getcwd()}/intrvw0{y_val}.zip', 'r')

    for i in range(4):
        if (y_val == 0) or (y_val == 1):
            fs = pd.read_csv(zf.open(fmli[i]), usecols= ID + FD + ND + DUR + TIME + DEMO + ASSETS, na_values='.')
            print(y_val, f'Q{i+1} dataset {fmli[i]} loaded; stimulus 2001')
        else:
            fs = pd.read_csv(zf.open(fmli[i]), usecols= ID + FD + ND + DUR + TIME + DEMO + DEMO2 + ASSETS, na_values='.') 
            print(y_val, f'Q{i+1} dataset {fmli[i]} loaded; stimulus 2008')

        fs['Year'] = 2000 + y_val
        fs['Quarter'] = f'Q{i+1}' # define the quarter to which the data refers to
        fs['CustID'] = fs['NEWID'].astype(str).str[:6] # First six digits of NEWID uniquely identify a person, the last digit refers to the interview
        fs['CustID'] = fs['CustID'].astype('int64')
        fs = fs.set_index(['CustID', 'QINTRVYR', 'QINTRVMO'])
        fs['FD'] = fs[FD].sum(axis = 1)
        fs['SND'] = fs[SND].sum(axis = 1)
        fs['ND'] = fs[ND].sum(axis = 1)
        fs['DUR'] = fs[DUR].sum(axis = 1)
        fs['valid_finassets'] = 0
        fs = fs.sort_index()
        fs.loc[fs['SAVA_CTX']=='D', 'valid_finassets'] = 1
        fs.loc[fs['CKBK_CTX']=='D', 'valid_finassets'] = 1
        fs.loc[fs['SECESTX_']=='D', 'valid_finassets'] = 1
        #fs['ASSETS'] = fs[ASSETS].sum(axis = 1)
        fs['finassets'] = fs[ASSETS].sum(axis = 1)
        fs.loc[fs['valid_finassets']==0,'finassets'] = np.nan
        fiscal[2000 + y_val,i+1] = fs

0 Q1 dataset intrvw00/intrvw00/fmli001x.csv loaded; stimulus 2001
0 Q2 dataset intrvw00/intrvw00/fmli002.csv loaded; stimulus 2001
0 Q3 dataset intrvw00/intrvw00/fmli003.csv loaded; stimulus 2001
0 Q4 dataset intrvw00/intrvw00/fmli004.csv loaded; stimulus 2001
1 Q1 dataset intrvw01/intrvw01/fmli011x.csv loaded; stimulus 2001
1 Q2 dataset intrvw01/intrvw01/fmli012.csv loaded; stimulus 2001
1 Q3 dataset intrvw01/intrvw01/fmli013.csv loaded; stimulus 2001
1 Q4 dataset intrvw01/intrvw01/fmli014.csv loaded; stimulus 2001
7 Q1 dataset intrvw07/fmli071x.csv loaded; stimulus 2008
7 Q2 dataset intrvw07/fmli072.csv loaded; stimulus 2008
7 Q3 dataset intrvw07/fmli073.csv loaded; stimulus 2008
7 Q4 dataset intrvw07/fmli074.csv loaded; stimulus 2008
8 Q1 dataset intrvw08/fmli081x.csv loaded; stimulus 2008
8 Q2 dataset intrvw08/fmli082.csv loaded; stimulus 2008
8 Q3 dataset intrvw08/fmli083.csv loaded; stimulus 2008
8 Q4 dataset intrvw08/fmli084.csv loaded; stimulus 2008
9 Q1 dataset intrvw09/fmli09

2.2: Append data for each year and each quarter for the two fiscal stimuli seperately

In [3]:
fs08_raw = fiscal[2007,1]

for y in list(range(7,10)):
    for q in list(range(1,5)):
        if (y == 7) and (q == 1):
            pass
        elif (y == 9) and (q > 1): #take only the first quarter of 2009
            pass
        else:
            fs08_raw = fs08_raw.append(fiscal[2000 + y,q])

fs01_raw = fiscal[2000,1]

for y in list(range(2)):
    for q in list(range(1,5)):
        if (y == 0) and (q == 1):
            pass
        else:
            fs01_raw = fs01_raw.append(fiscal[2000 + y,q])

fiscal_stimulus = dict()
fiscal_stimulus[1] = fs01_raw
fiscal_stimulus[8] = fs08_raw


**Step 3**: Generate rebate data 

3.1. Fiscal stimulus 2008

3.1.1 Load data

In [4]:
REBATE = ['RBTMO', 'RBTAMT', 'CHCKEFT']

zf = zipfile.ZipFile(f'{os.getcwd()}/intrvw08.zip', 'r') # rebate data is only in the 2008 folder
with zf as zipobj:
    df = zipobj.namelist()
rbt = [s for s in df if 'rbt08' in s] # relevant file of the previously downloaded data
zf = zipfile.ZipFile(f'{os.getcwd()}/intrvw08.zip', 'r')
rebate_fs08_raw = pd.read_csv(zf.open(rbt[0]), usecols = REBATE +  ['CUID'] + ['NEWID'] + ['USDINTMO'] + ['USDINTYR'],na_values='.')
print(rebate_fs08_raw.isin([0]).sum()) #no zero in data frame


NEWID       0
CUID        0
RBTMO       0
RBTAMT      0
CHCKEFT     0
USDINTMO    0
USDINTYR    0
dtype: int64


3.1.2. Clean rebate data for fs 2018

In [5]:
#Sum across those individuals who have two entries in the same month. 
#In order not to lose any information on the way the rebate was received, generate variables that capture the amount received electronically or per mail.

rebate_fs08 = rebate_fs08_raw.set_index(['NEWID','CHCKEFT'])
rebate_fs08.loc[:,'rbtamt_chk'] = 0
rebate_fs08.loc[:,'rbtamt_e'] = 0 # generate new variables that capture the amount received electronically/per mail

rebate_fs08.loc[pd.IndexSlice[:,2.0],'rbtamt_e'] = rebate_fs08.loc[pd.IndexSlice[:,2.0],'RBTAMT'] 
rebate_fs08.loc[pd.IndexSlice[:,1.0],'rbtamt_chk'] = rebate_fs08.loc[pd.IndexSlice[:,1.0],'RBTAMT']

rebate_fs08 = rebate_fs08.groupby(['CUID', 'NEWID', 'RBTMO']).agg({  
                         #'USDINTMO':['first'], #the first entry is always the same as the last
                         #'USDINTYR':['first'], #the first entry is always the same as the last                                         
                         'RBTAMT': ['sum'], # sum over entries in the same month, the same interview and the same individual
                         'rbtamt_chk': ['sum'],
                         'rbtamt_e': ['sum']}) 

rebate_fs08.columns = rebate_fs08.columns.droplevel(1) #It is not necessary to keep the column labels generated by the groupby command
rebate_fs08.loc[(rebate_fs08['rbtamt_chk']==0) & (rebate_fs08['rbtamt_e']==0), ['rbtamt_e','rbtamt_chk']] = np.nan 
        #entries with no info on how received should be nan




rebate_fs08 = pd.merge(rebate_fs08.reset_index(), fs08_raw.reset_index().loc[:,TIME + ['NEWID']] ,on='NEWID', how='inner' ) #merge info on timing of interview
rebate_fs08['diff'] = rebate_fs08['QINTRVMO'] - rebate_fs08['RBTMO'] + (rebate_fs08['QINTRVYR'] - 2008)*12 
    #difference between interview time and rebate receipt
print('no. of obs. where difference between timing of rebate and interview is either 0 or larger than 3:',
      rebate_fs08['NEWID'].loc[(rebate_fs08['diff'] == 0 ) | (rebate_fs08['diff']>3)].count())
print('How is diff between interview time and rebate receipt distributed?')
print(rebate_fs08.groupby('diff')['NEWID'].count()) #In theory the diff should be in range of 1-3


##################
#when the difference is 0, the reference for the rebate should be changed to the later interview
print('no. of obs. where the difference is zero:', rebate_fs08.loc[rebate_fs08['diff']==0, 'CUID'].count()) 
rebate_fs08['intnr'] = rebate_fs08['NEWID'].astype(str).str[-1:].astype(int) # last number of newid refers to the interview number
print('no. of obs. where the difference is zero and it is the last interview:',
      rebate_fs08.loc[(rebate_fs08['diff']==0) & (rebate_fs08['intnr']==5), 'CUID'].count())

#rebate_fs08 = rebate_fs08.set_index('CUID')

RBT = ['RBTAMT', 'rbtamt_chk', 'rbtamt_e']
LAGRBT = ['last_' + var for var in RBT]
LAGRBT = [item.lower() for item in LAGRBT] #lagged variables
FUTRBT = ['fut_' + var for var in RBT]
FUTRBT = [item.lower() for item in FUTRBT] #future variables

rebate_fs08['int_amount'] =  rebate_fs08.groupby('CUID')['NEWID'].transform('count') #generate sum of entries for given individual

for i in range(len(RBT)):
    rebate_fs08[LAGRBT[i]] = rebate_fs08.sort_values(by=['NEWID']+TIME).groupby('CUID')[RBT[i]].shift(1) #generate lag
    rebate_fs08[FUTRBT[i]] = rebate_fs08.sort_values(by=['NEWID']+TIME).groupby('CUID')[RBT[i]].shift(-1) #generate future
    rebate_fs08.loc[(rebate_fs08['int_amount']==1) & (rebate_fs08['diff']==0), FUTRBT[i]] = rebate_fs08.loc[(rebate_fs08['int_amount']==1)
                                                                                                            & (rebate_fs08['diff']==0), RBT[i]]
                                                                                            
rebate_fs08.loc[(rebate_fs08['int_amount']==1) & (rebate_fs08['diff']==0), RBT] = 0 #one entry for person and diff=0: change rebate from now to future

print('no. of obs. where the difference is zero and there is more than one observation per interview:',
      rebate_fs08.loc[rebate_fs08['NEWID'].isin(rebate_fs08.loc[(rebate_fs08['int_amount']>1) & (rebate_fs08['diff']==0),'NEWID']),
                      'NEWID'].count())
rebate_fs08 = rebate_fs08.reset_index()
print('no. of obs. where the difference is zero and there is more than one observation per individual:',
      rebate_fs08.loc[rebate_fs08['CUID'].isin(rebate_fs08.loc[(rebate_fs08['int_amount']>1) & (rebate_fs08['diff']==0),'CUID']),
                      'NEWID'].count())
    #the no. of obs. is identical between interviews and individuals. Actually, for each individual with int_amount>1 and diff==0 the second 
    #(and always last) entry is always the one that has a diff of 0. 
    #Hence, just drop those entries bc the amount is already captured in the future_rbtamt of the entry before
rebate_fs08 = rebate_fs08.set_index('CUID')
rebate_fs08['dropobs'] = 0 #generate help variable
rebate_fs08.loc[rebate_fs08['NEWID'].isin(rebate_fs08.loc[(rebate_fs08['int_amount']>1) & (rebate_fs08['diff']==0),'NEWID']) & 
                rebate_fs08['fut_rbtamt'].isna(), 'dropobs'] = 1 
    #where there are more than one entries for rebates but 1 has a diff = 0; dropobs=1 for diff=0
rebate_fs08 = rebate_fs08.loc[rebate_fs08['dropobs']==0] #drop those observations

##################
#when the difference is 4, the reference for the rebate should be changed to the previous interview
print('no. of obs. where the difference is 4 and it is the first interview:', rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']==2) , 'NEWID'].count())
print('no. of obs. where the difference is 4,it is the first interview and there is exactly one entry per individual:',
      rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']==2) & (rebate_fs08['int_amount']==1) , 'NEWID'].count())
print('no. of obs. where the difference is 4, it is the first interview and there is more than one entry per individual:',
      rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']==2) & (rebate_fs08['int_amount']>1) , 'NEWID'].count())
print('unique no. of obs. where the difference is 4, it is the first interview and there is more than one entry per individual:',
      rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']==2) & (rebate_fs08['int_amount']>1) , 'NEWID'].nunique())
#recall that the if the diff is 4 the rebate should be captured as received in the last period. For those individuals where we have more than one entry and the
#diff is 4 the lower number entries already capture the last_rbtamt. Hence, we can just delete those entries. 
rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']==2) & (rebate_fs08['int_amount']>1), 'dropobs'] = 1
rebate_fs08 = rebate_fs08.loc[rebate_fs08['dropobs']==0] #drop those observations

rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']==2) & (rebate_fs08['int_amount']==1), 
                'last_rbtamt']= rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']<3), 'RBTAMT']
rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']==2) & (rebate_fs08['int_amount']==1), 
                'last_rbtamt_e'] = rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']<3), 'rbtamt_e']
rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']==2) & (rebate_fs08['int_amount']==1), 
                'last_rbtamt_chk'] = rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']<3), 'rbtamt_chk']

rebate_fs08.loc[(rebate_fs08['diff']==4) & (rebate_fs08['intnr']<3), ['RBTAMT','rbtamt_chk','rbtamt_e']] = 0


##################
#when the difference is >4, the observations should be dropped
rebate_fs08 = rebate_fs08.loc[rebate_fs08['diff']<5].drop(columns=['dropobs','QINTRVMO','QINTRVYR']) #drop any observation where diff==5 or higher
rebate_fs08.loc[rebate_fs08['NEWID']==1867435]

#################
#change to single entry for interview for a given individual
rebate_fs08[LAGRBT + FUTRBT] = rebate_fs08[LAGRBT + FUTRBT].fillna(0) #replace all missing values with zero. There are no missing values in the initial data frame

rebate_fs08['rbtnr_perint'] = 1
rebate_fs08['rbtnr_perint'] = rebate_fs08.groupby('NEWID')['rbtnr_perint'].transform('sum') #count number of entries per interview
print('The number of individuals who have more than one entry per interview is',
     rebate_fs08.loc[rebate_fs08['rbtnr_perint']>1,'NEWID'].count())
print('Who has more than 2 entries per interview?')
print(rebate_fs08.loc[rebate_fs08['rbtnr_perint']>2,['NEWID','RBTAMT']])
columns = list(rebate_fs08.columns.values)

#aggregate over interviewid (interviews)
columns =['rbtnr_perint','CUID','RBTMO'] + RBT + LAGRBT + FUTRBT  
agg = ['first','first'] + [['first','last']]*len(['RBTMO'] + RBT) + ['first'] * len(LAGRBT) + ['last']*len(FUTRBT)  
columns_dict = dict(zip(columns, agg))
rebate_fs08 = rebate_fs08.reset_index().sort_values(by=['RBTMO']).groupby('NEWID').agg(columns_dict)
        #agg: for rbtnr_perint and CUID the first entry; rbtmo and rbt the first and last; lagrbt the first; futrbt the last

#rename columns
columns = list(zip(RBT + ['RBTMO'] , ['first']*len(RBT + ['RBTMO']))) + list(zip(RBT + ['RBTMO'], ['last']*len(RBT + ['RBTMO'])))
rename = [x + '_1' for x in RBT + ['RBTMO']] + [x + '_2' for x in RBT + ['RBTMO']]
rename = [item.lower() for item in rename]
for i in range(len(columns)):
    rebate_fs08[rename[i]] = rebate_fs08[columns[i]] #duplicate columns
    
rebate_fs08 = rebate_fs08.drop(columns=columns)
rebate_fs08.columns = rebate_fs08.columns.droplevel(1)
rebate_fs08 = rebate_fs08.rename(columns={'RBTAMT':'rbtamt'})

rename = rename[4:]
for i in range(len(rename)):
    rebate_fs08.loc[rebate_fs08['rbtnr_perint'] == 1, rename[i]] = 0 #if there's one entry/interview, the rbtamt_2 etc. should be zero

rebate = ['rbtamt', 'rbtamt_chk', 'rbtamt_e']
for i in range(len(rebate)):
    rebate_fs08[rebate[i]] = rebate_fs08[[rebate[i]+'_1',rebate[i]+'_2']].sum(axis = 1) #sum over rbtamt entries/interview
    
rebate_fs08.loc[1887894,'rbtamt'] = 2400 #two individuals have three entries/interview. Change manually aggregated rbtamt/interview
rebate_fs08.loc[1887894,'rbtamt_chk'] = 2400
rebate_fs08.loc[1907123,'rbtamt'] = 1200
rebate_fs08.loc[1907123,'rbtamt_chk'] = 300
rebate_fs08.loc[1907123,'rbtamt_e'] = 900

rebate_fs08.loc[(rebate_fs08['rbtamt_e_1'].isna()) & (rebate_fs08['rbtamt_chk_1'].isna()), ['rbtamt_chk','rbtamt_e']] = np.nan


no. of obs. where difference between timing of rebate and interview is either 0 or larger than 3: 775
How is diff between interview time and rebate receipt distributed?
diff
0       94
1     1784
2     1719
3     1631
4      667
12       2
13       9
14       3
Name: NEWID, dtype: int64
no. of obs. where the difference is zero: 94
no. of obs. where the difference is zero and it is the last interview: 94
no. of obs. where the difference is zero and there is more than one observation per interview: 17
no. of obs. where the difference is zero and there is more than one observation per individual: 17
no. of obs. where the difference is 4 and it is the first interview: 667
no. of obs. where the difference is 4,it is the first interview and there is exactly one entry per individual: 641
no. of obs. where the difference is 4, it is the first interview and there is more than one entry per individual: 26
unique no. of obs. where the difference is 4, it is the first interview and there is more t

3.2 For fiscal stimulus 2001, take data from Johnson, Parker, Souleles. Extracted from https://www.openicpsr.org/openicpsr/project/116245/version/V1/view

In [6]:
rebate_fs08.loc[rebate_fs08['rbtamt']>0]

,rbtnr_perint,CUID,last_rbtamt,last_rbtamt_chk,last_rbtamt_e,fut_rbtamt,fut_rbtamt_chk,fut_rbtamt_e,rbtamt_1,rbtamt_chk_1,rbtamt_e_1,rbtmo_1,rbtamt_2,rbtamt_chk_2,rbtamt_e_2,rbtmo_2,rbtamt,rbtamt_chk,rbtamt_e
NEWID,,,,,,,,,,,,,,,,,,,
1838725,1,183872,0.0,0.0,0.0,0.0,0.0,0.0,1200,0.0,1200.0,4,0,0.0,0.0,0,1200,0.0,1200.0
1839135,1,183913,0.0,0.0,0.0,0.0,0.0,0.0,1200,0.0,1200.0,5,0,0.0,0.0,0,1200,0.0,1200.0
1839205,1,183920,0.0,0.0,0.0,0.0,0.0,0.0,1200,0.0,1200.0,5,0,0.0,0.0,0,1200,0.0,1200.0
1839445,1,183944,0.0,0.0,0.0,0.0,0.0,0.0,900,900.0,0.0,5,0,0.0,0.0,0,900,900.0,0.0
1839535,1,183953,0.0,0.0,0.0,0.0,0.0,0.0,1200,0.0,1200.0,4,0,0.0,0.0,0,1200,0.0,1200.0
1839595,1,183959,0.0,0.0,0.0,0.0,0.0,0.0,600,0.0,600.0,5,0,0.0,0.0,0,600,0.0,600.0
1839615,1,183961,0.0,0.0,0.0,0.0,0.0,0.0,1500,0.0,1500.0,5,0,0.0,0.0,0,1500,0.0,1500.0
1839815,1,183981,0.0,0.0,0.0,600.0,600.0,0.0,300,0.0,300.0,5,0,0.0,0.0,0,300,0.0,300.0
1839855,1,183985,0.0,0.0,0.0,0.0,0.0,0.0,1200,0.0,1200.0,5,0,0.0,0.0,0,1200,0.0,1200.0


In [7]:
JPS = os.getcwd() +'\\JPSAERdata.dta'
JPS
#JPSAERdata
rebate_fs01 = pd.read_stata(JPS)
rebate_fs01 = rebate_fs01.loc[:,['newid', 'intview', 'yymm', 'taxreb', 'itaxreb', 'ltaxreb', 'iltaxreb', 'l2taxreb', 'il2taxreb', 'itotalreb']] #relevant variables

rebate_fs01 = rebate_fs01.rename(columns={'newid': 'CustID'}) #authors' variable newid is custid in all other datasets
rebate_fs01['NEWID'] = [str(x) +  str(y) for x, y in zip(rebate_fs01['CustID'], rebate_fs01['intview'])] #generate newid as a combination of custid and the number of the interview
rebate_fs01 = rebate_fs01.set_index('NEWID')


**Step 4: load mortgage data**

In [8]:
MORTGAGE = ['QESCROWX', 'QBLNCM1X', 'ORGMRTX', 'NEWMRRT', 'QMRTTERM', 'FIXEDRTE', 'OWNYF', 'FRSTPYYR', 'FRSTPYMO'] 
    #last escrow payment; outstanding principal balance at beginning of month M1; mortage amount; current interest rate;
    #term of loan in years; fixed rate mortgage?; property code (100 = home currently lived in); year of first payment; month of first payment
    
y = [0,1,7,8,9]

mortgage = dict()
for y_val in y:
    zf = zipfile.ZipFile(f'{os.getcwd()}/intrvw0{y_val}.zip', 'r')
    with zf as zipobj:
        df = zipobj.namelist()
    mor = [s for s in df if 'mor' in s]
    zf = zipfile.ZipFile(f'{os.getcwd()}/intrvw0{y_val}.zip', 'r')
    morg = pd.read_csv(zf.open(mor[0]), usecols= ['NEWID'] + ['QYEAR'] + MORTGAGE, na_values='.' ) # , dtype=types_mortgage
    print(y_val, f'dataset {mor} loaded')
    morg = morg.loc[morg['QYEAR']< 20011 + y_val*10]
    mortgage[2000 + y_val] = morg

0 dataset ['intrvw00/expn00/mor00.csv'] loaded
1 dataset ['intrvw01/expn01/mor01.csv'] loaded
7 dataset ['expn07/mor07.csv'] loaded
8 dataset ['expn08/mor08.csv'] loaded
9 dataset ['expn09/mor09.csv'] loaded


In [9]:

morg08_raw = mortgage[2007]

for y in list(range(7,10)):
    if (y == 7):
        pass
    else:
        morg08_raw = morg08_raw.append(mortgage[2000 + y])

morg01_raw = mortgage[2000]

for y in list(range(2)):
    if (y == 0):
        pass
    else:
        morg01_raw = morg01_raw.append(mortgage[2000 + y])

mortgage_stimulus = dict()

mortgage_stimulus[1] = morg01_raw
mortgage_stimulus[8] = morg08_raw

Clean up data and generate variables

In [10]:
y = [1,8]

for y_val in y:
    print(f'The number of duplicates in the data frame (year {2000+y_val}):', mortgage_stimulus[y_val].loc[mortgage_stimulus[y_val].duplicated()==True, 'NEWID'].count()) 
    #For 2008: 42 duplicate obs.; note without controlling for years 6775 obs are duplicates, when only keeping the first entry we still have 6065 duplicates. 
    #Even though unlikely, it could be that the 42 duplicate entries are actually two seperate mortgages; keep them for now. 

    mortgage_stimulus[y_val] = pd.merge(mortgage_stimulus[y_val].reset_index() ,fiscal_stimulus[y_val].reset_index().loc[:,TIME + ['NEWID']],on='NEWID', how='inner') 
        # merge with info on timing of the interview

    mortgage_stimulus[y_val]['time'] = ((mortgage_stimulus[y_val]['QINTRVYR'] - mortgage_stimulus[y_val]['FRSTPYYR'])*12 +  
                                        mortgage_stimulus[y_val]['QINTRVMO'] - mortgage_stimulus[y_val]['FRSTPYMO'])/12
    mortgage_stimulus[y_val]['timeleft'] = mortgage_stimulus[y_val]['QMRTTERM'] - mortgage_stimulus[y_val]['time']
    mortgage_stimulus[y_val]['nmorg'] = 1 # generate variable that counts the number of mortgages

    #mortgage_stimulus[y_val].reset_index().set_index('NEWID').loc[1993712,:]

    #QESCROWX QBLNCM1X ORGMRTX nmort

    mortgage_stimulus[y_val] = mortgage_stimulus[y_val].groupby(['NEWID']).agg({  
                             'timeleft':['max'], #use info until individual does not have to pay off any debt
                             'time':['median'], # take median value of time elapsed since first mortgage payment
                             'NEWMRRT':['median'], #take median value of current interest rate 
                             'nmorg':['sum'], #capture the number of mortgages by individual                                         
                             'QINTRVYR': ['first','last'], #keep information on year of interview
                             'QINTRVMO': ['first', 'last'], #keep information on month of interview
                             'QESCROWX': ['sum'], #sum all amounts of last regular escrow payment
                             'QBLNCM1X': ['sum'], #sum all principal balance outstanding at the beginning of month M1
                             'ORGMRTX': ['sum'], #sum all amount of mortgage
                                }) 
    mortgage_stimulus[y_val] = mortgage_stimulus[y_val].reset_index()
    print('Number of entries where first entry of interview year is not the same as the last:',
          mortgage_stimulus[y_val].loc[(mortgage_stimulus[y_val]['QINTRVYR','first']!=mortgage_stimulus[y_val]['QINTRVYR','last']), 'NEWID'].count())
    
    print('Number of entries where first entry of interview month is not the same as the last:',
          mortgage_stimulus[y_val].loc[(mortgage_stimulus[y_val]['QINTRVMO','first']!=mortgage_stimulus[y_val]['QINTRVMO','last']), 'NEWID'].count())
    
    mortgage_stimulus[y_val] = mortgage_stimulus[y_val].set_index('NEWID')
    mortgage_stimulus[y_val] = mortgage_stimulus[y_val].drop(columns=[['QINTRVYR','first'],['QINTRVYR','last'],['QINTRVMO','first'],['QINTRVMO','last']])

    mortgage_stimulus[y_val].columns = mortgage_stimulus[y_val].columns.droplevel(1) # drop column labels generated by the groupby command
    mortgage_stimulus[y_val] = mortgage_stimulus[y_val].rename(columns={'QESCROWX': 'qescrowx_sum', 'QBLNCM1X': 'qblncm1x_sum', 'ORGMRTX': 'orgmrtx_sum', 'NEWMRRT': 'newmrrt_median'})
    mortgage_stimulus[y_val]['morgpayment'] = mortgage_stimulus[y_val]['qblncm1x_sum']*mortgage_stimulus[y_val]['newmrrt_median']/ mortgage_stimulus[y_val]['timeleft'] /12
        #morgage payment per month: sum of outstanding balance times median interest rate on morgage divided by max time left on morgage times 12
    mortgage_stimulus[y_val].loc[mortgage_stimulus[y_val]['timeleft']==0, 'morgpayment'] = mortgage_stimulus[y_val]['qblncm1x_sum']*mortgage_stimulus[y_val]['newmrrt_median']
morg01_raw = mortgage_stimulus[1]
morg08_raw = mortgage_stimulus[8]

The number of duplicates in the data frame (year 2001): 29
Number of entries where first entry of interview year is not the same as the last: 0
Number of entries where first entry of interview month is not the same as the last: 0
The number of duplicates in the data frame (year 2008): 42
Number of entries where first entry of interview year is not the same as the last: 0
Number of entries where first entry of interview month is not the same as the last: 0


**Step 5: Merge the data sets 2008**

In [14]:
fs08 = pd.merge(fs08_raw.reset_index(), rebate_fs08.reset_index(), how = 'left', on = 'NEWID') #.set_index([ 'CustID','QINTRVYR', 'QINTRVMO']).sort_index()


fs08 = fs08.reset_index().set_index(['CustID']) #.drop(columns='CUID')

#keep only observations for individuals who were questioned at least once between 6,2008 and 3,2009
fs08['intrvyr_max'] =  fs08.groupby('CustID')['QINTRVYR'].transform('last')
fs08['intrvmo_max'] =  fs08.groupby('CustID')['QINTRVMO'].transform('last')

fs08['keepcons'] = 0

fs08.loc[(fs08['intrvyr_max']==2008) & (fs08['intrvmo_max']>=6),['keepcons']] = 1 
#fs08.loc[(fs08['QINTRVYR']==2009) & (fs08['QINTRVMO']<=3),['keepcons']] = 1
fs08.loc[(fs08['intrvyr_max']==2009) & (fs08['intrvmo_max']<=3),['keepcons']] = 1 
fs08 = fs08.loc[fs08['keepcons']==1]

fs08.dtypes #check datatypes of variables in the data frame
fs08['age'] = (fs08['AGE2'] + fs08['AGE_REF'])/2 #where we have information about husband and spouse age is a combination of both
fs08.loc[fs08['age'].isna(), 'age'] = fs08.loc[fs08['age'].isna(), 'AGE_REF'] #where no info about partner's age, age is just ref person's age
fs08['adults'] = fs08['FAM_SIZE'] - fs08['PERSLT18'] #number of adults
fs08['timetrend'] = (fs08['QINTRVYR'] - 2008)*12 + fs08['QINTRVMO']
fs08['int_entries'] = 1
fs08['int_entries'] = fs08.groupby('NEWID')['int_entries'].transform('sum')

#There are some variables for which lagged values are required
LAGVAR = ['age', 'PERSLT18', 'adults', 'timetrend', 'FD', 'SND', 'ND', 'DUR']
LAGNAMES = ['last_' + var for var in LAGVAR]
CHGNAMES = ['chg_' + var for var in LAGVAR]

#Augment lagged and future values of rebate
RBT = ['rbtamt', 'rbtamt_chk', 'rbtamt_e']
LAGRBT = ['last_' + var for var in RBT] #lagged variables
FUTRBT = ['fut_' + var for var in RBT] #future variables

for i in range(len(RBT)):
    fs08.loc[fs08[RBT[i]]==0, RBT[i]] = np.nan #change from zero to Nan (as previously checked there are no zeros in the original data)
    fs08.loc[fs08[LAGRBT[i]]==0, LAGRBT[i]] = np.nan
    fs08.loc[fs08[FUTRBT[i]]==0, FUTRBT[i]] = np.nan
    #just one entry left per interview, aggregate over custid to add lag/future value of rebate if it's zero:
    fs08.loc[(fs08[LAGRBT[i]]==0) | (fs08[LAGRBT[i]].isna()),
             LAGRBT[i]] =  fs08.loc[(fs08[LAGRBT[i]]==0) | (fs08[LAGRBT[i]].isna())].groupby('CustID')[RBT[i]].shift(1) 
    fs08.loc[(fs08[FUTRBT[i]]==0) | (fs08[FUTRBT[i]].isna()),
             FUTRBT[i]] =  fs08.loc[(fs08[FUTRBT[i]]==0) | (fs08[FUTRBT[i]].isna())].groupby('CustID')[RBT[i]].shift(-1) 
    
 
fs08 = fs08.reset_index()

for i in range(len(LAGVAR)):
    fs08[LAGNAMES[i]] = fs08.sort_values(by = ['NEWID']).groupby('CustID')[LAGVAR[i]].shift(1)
    fs08[CHGNAMES[i]] = fs08[LAGVAR[i]] - fs08[LAGNAMES[i]]
    if LAGVAR[i] == 'age':
        fs08.loc[abs(fs08[CHGNAMES[i]])>1, 'keepcons'] = 0 #age of reference person shouldn't change by more than 1 between the interviews
    elif LAGVAR[i] == 'timetrend':
        fs08.loc[abs(fs08[CHGNAMES[i]]>3), 'keepcons'] = 0 #time between interviews shouldn't be larger than 1
        fs08.loc[(fs08[CHGNAMES[i]]==0) & (fs08['int_entries']<2), 'keepcons'] = 0 
        #the time between two interviews shouldn't be zero, if there are two entries per interview the lag will be zero; make sure to not drop  
    elif (LAGVAR[i] == 'PERSLT18') | (LAGVAR[i] == 'adults') :
        fs08.loc[abs(fs08[CHGNAMES[i]])>3, 'keepcons'] = 0 #change in number of adults or individuals below 18 shouldn't change by more than 3
    else:
        pass
    
#fs08['keepcons'] = fs08.groupby('CustID')['keepcons'].transform('min')
fs08 = fs08.loc[fs08['keepcons']==1] #clean data
fs08 = fs08.drop(columns = LAGNAMES[0:3] + CHGNAMES[0:3] + ['keepcons', 'index'])


fs08 = fs08.loc[fs08['age'].between(21,85)] #drop very old and very young households

fs08 = fs08.loc[fs08['ST_HOUS'] != 1] #drop households with student housing
fs08['totalint'] =  fs08.groupby('CustID')['QINTRVMO'].transform('count')
fs08 = fs08.loc[fs08['totalint']>1] #drop households with only one interview entry

#merge with mortgage data
fs08 = pd.merge(fs08.reset_index(), morg08_raw.reset_index(), how = 'left', on = 'NEWID').sort_values(by = ['NEWID']).set_index('CustID') #27330 obs
fs08 = fs08.drop(columns = 'index')


##drop first interview
#fs08['drop_first'] = 1
#fs08['drop_first'] = fs08.sort_values(by=['NEWID']).groupby('CustID')['drop_first'].transform('cumsum')
#for i in range(len(CHGNAMES[4:])):
#    fs08.loc[(fs08['drop_first']==1)&(fs08[CHGNAMES[4:][i]].notna()),'drop_first']=2
#
#   
#fs08 = fs08.loc[fs08['drop_first']>1].drop(columns='drop_first')

fs08.to_csv( os.getcwd() + '\\fs08.csv')

-1-------------------------------------------
5023
--------------------------------------------
-2-------------------------------------------
4893
--------------------------------------------
-2-------------------------------------------
4339
--------------------------------------------
